# Evaluating Scene Recognition Classifier

In [ ]:
import torch
import logging

from recognizer.data import ProjectDataset
from recognizer.utils.visualise import Visualise
from recognizer.model import ResNetFineTunedClassifier
from recognizer.trainer import TorchRunner
from recognizer.utils.types import EvalResults

In [ ]:
# configuring logging level
logging.basicConfig(level=logging.INFO)
# picking appropriate device to train the model
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
# dataset related config
datasets_dir = "../data"
training_dataset = f"{datasets_dir}/train"
testing_dataset = f"{datasets_dir}/test"
DEBUG_MODE = False

In [ ]:
train_loader, val_loader, test_loader = ProjectDataset.get_loaders()

In [ ]:
if DEBUG_MODE:
    Visualise.display_sample_per_class(test_loader)

In [ ]:
classifier = ResNetFineTunedClassifier()
classifier.load("../checkpoints/models/latest.safetensors")
TorchRunner.get_summary(classifier, (3, 244, 244))

## Test

In [ ]:
if DEBUG_MODE:
    _, _ = TorchRunner.test(
        loader=test_loader,
        model=classifier,
        loss_fn=torch.nn.functional.cross_entropy,
        device=device,
    )

test_results = TorchRunner.evaluate(
    loader=test_loader, model=classifier, k=5, device=device
)

In [ ]:
top_1_cm, top_1_precision, top_1_recall, top_1_f1 = test_results.compute_top_1_scores()
logging.info(f"\nPrecision: {top_1_precision}\nRecall: {top_1_recall}\nF1: {top_1_f1}")
Visualise.plot_confusion_matrix(top_1_cm, size=(20, 17), cmap="Greens")

In [ ]:
top_k_cm, top_k_precision, top_k_recall, top_k_f1 = test_results.compute_top_k_scores()
logging.info(f"\nPrecision: {top_k_precision}\nRecall: {top_k_recall}\nF1: {top_k_f1}")
Visualise.plot_confusion_matrix(top_k_cm, size=(20, 17), cmap="Greens")

In [ ]:
Visualise.show_results(eval_results=test_results, n=20)